In [ ]:
import pandas as pd
from pandas import DataFrame
from typing import Any

df = pd.read_csv("../../docs/talent.csv")
df.head(2)

,email,name,skills,project,project_details,project_skills
0,thomas.nelson@test.org,Thomas Nelson,"Security, Pandas, Go",Data Analytics,A data analytics project transforming raw data...,"Pandas,Tableau,Big Data, Scala,Spark"
1,lucy.clark@test.org,Lucy Clark,"WordPress, Scrum, Go, SQL, Linux",UI Design,"A UI design project creating intuitive, user-f...","CSS3,Angular,HTML5,Express.js,JavaScript"


In [ ]:
# modification is required
# data lose while returning the final output
def convert_df(input: Any, col:str, splitBy = None)->DataFrame|None: 
    df1 = pd.DataFrame(input)
    df1 = df1[[col]]
    col_trans = 'transformed'
    if col in df1.columns:
        if splitBy is not None:
            df1[col_trans] =  df1[col].str.split(splitBy)
            df1[col_trans] = df1[col_trans].explode(col_trans).drop_duplicates().reset_index(drop=True)  
        else:
            df1[col_trans] = df1[col].drop_duplicates().reset_index(drop=True)
            df1 = df1.dropna()
    else:
        return None
    return df1


In [ ]:
data = convert_df(df,'skills',',')
data

In [7]:
from langchain_openai import AzureChatOpenAI,AzureOpenAIEmbeddings
import os
from dotenv import load_dotenv

load_dotenv()

llm = AzureChatOpenAI(  
    azure_deployment="gpt-4.1",  # or your deployment
    api_version="2024-12-01-preview",  # or your api version
    temperature=0,
    max_tokens=None,
    timeout=None,
    max_retries=2,
)

embedding = AzureOpenAIEmbeddings(
    azure_deployment="text-embedding-3-small",
    api_version="2024-12-01-preview"
)

In [ ]:

def embed_df(input:Any)->DataFrame|None:
    if input is not None:
        df = pd.DataFrame(input)
        df["embedding"] = df['transformed'].apply(lambda item: embedding.embed_query(item[0]))
        return df 
    else:
        return None



In [ ]:
response = embed_df(data)

In [ ]:
def split_dataframe(df, chunk_size = 5):
    chunks = list()
    num_chunks = len(df) # chunk_size + 1
    for i in range(num_chunks):
        chunks.append(df[i*chunk_size:(i+1)*chunk_size])
    return chunks

In [8]:
#neo4j Driver
from langchain_neo4j import GraphCypherQAChain,Neo4jGraph
from neo4j import RoutingControl

def db()->Neo4jGraph|None:
    return Neo4jGraph(
    url=os.getenv('NEO4J_URI'),
    username=os.getenv('NEO4J_USERNAME'),
    password=os.getenv('NEO4J_PASSWORD'),
    database='neo4j'
    )

In [ ]:
resp1 = db().query(
    """
    MATCH (n) RETURN COUNT(n) as Count
    """
)

In [ ]:
db().query(
    'CREATE CONSTRAINT Name_Skill IF NOT EXISTS FOR (n:Skill) REQUIRE (n.name) IS NODE KEY'
)

In [ ]:
db().query(
    'CREATE CONSTRAINT Name_Person IF NOT EXISTS FOR (n:Person) REQUIRE (n.email) IS NODE KEY'
)

In [ ]:
result = db().query("SHOW CONSTRAINTS")
len(result)

In [ ]:
df_person = pd.DataFrame(df)
df_person['skills'] = df_person['skills'].str.split(',')
df_person.head()

In [ ]:
driver = db()
with driver:
    for index,item in df_person.iterrows():
        properties={
            "rows":{
                'email':item['email'],
                'name' : item['name'],
                'skills' : item['skills']
            }
        }
        driver.query(
            """
                UNWIND $rows as row
                MERGE (p:Person {email:row.email})
                set p.name = row.name
                with p, row
                foreach (item in row.skills |  
                        MERGE(s:Skill {name: rtrim(ltrim(item))}) 
                        MERGE (p)-[:KNOWS]->(s))
            """, properties
        )   
  
    

In [23]:
#add project node to db and map to person
driver = db()
with driver:
    for row,item in df.iterrows():
        properties = {
            'rows':{
                'email': item['email'],
                'project':item['project'],
                'project_details':item['project_details']
            }
        }
        driver.query(
            """
            UNWIND $rows as row
            MERGE(p:Project{name:row.project})
            set p.project_details = row.project_details
            with row, p
                MATCH(p1:Person{email: row.email})
                    MERGE(p1)-[:ASSIGNED_TO]->(p)

            """, properties
        )

In [16]:
df_projects = pd.DataFrame(df)
df_projects['project_skills'] = df_projects['project_skills'].str.split(',')
df_projects.head()

,email,name,skills,project,project_details,project_skills
0,thomas.nelson@test.org,Thomas Nelson,"Security, Pandas, Go",Data Analytics,A data analytics project transforming raw data...,"[Pandas, Tableau, Big Data, Scala, Spark]"
1,lucy.clark@test.org,Lucy Clark,"WordPress, Scrum, Go, SQL, Linux",UI Design,"A UI design project creating intuitive, user-f...","[CSS3, Angular, HTML5, Express.js, JavaScript]"
2,richard.jackson@test.org,Richard Jackson,"System Design, PyTorch, Express.js, DevOps",DevOPS,"A DevOps project automating development, testi...","[CI/CD, DevOps, Agile, Cloud Architecture, Ang..."
3,amelia.hall@test.org,Amelia Hall,"Agile, CSS3, R, Azure",UI Design,"A UI design project creating intuitive, user-f...","[CSS3, Angular, HTML5, Express.js, JavaScript]"
4,david.hill@test.org,David Hill,"Java, Scrum, Angular",Other Projects,"An IT project delivering secure, scalable, and...","[CI/CD, Agile, JavaScript, Blockchain, SQL, Cy..."


In [15]:
driver = db()
with driver:
    for index,item in df_projects.iterrows():
        properties={
            "rows":{
                'project_name' : item['project'],
                'project_skills' : item['project_skills']
            }
        }
        driver.query(
            """
                UNWIND $rows as row
                MATCH (p:Project {name:row.project_name})
                with p, row.project_skills as skills
                    unwind skills as skill
                    MATCH(s:Skill {name:skill})
                    MERGE (p)-[:IMPLEMENTS]->(s)
            """, properties
        )   
  
    